## **1. Charger et Préparer le Dataset**

In [25]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('/content/entrainement_chatbot.csv')
# Supprimer les lignes où 'Question' est manquant
df = df.dropna(subset=['Question'])
# Afficher un aperçu des données
print(df.head())


          Categorie            Tag  \
0  Droit du travail  contrat_stage   
1  Droit du travail  contrat_stage   
2  Droit du travail  contrat_stage   
3  Droit du travail  contrat_stage   
4  Droit du travail  contrat_stage   

                                            Question  \
0  Quel est l'objectif principal du contrat de st...   
1    Qu'est-ce qu'un contrat de stage selon la loi ?   
2  Quel type d'expérience le contrat de stage vis...   
3       Quand doit être conclu le contrat de stage ?   
4  Quel est le format requis pour le contrat de s...   

                                             Réponse  
0  L'objectif du contrat de stage est de favorise...  
1  Un contrat de stage est une convention par laq...  
2  Le contrat de stage vise à faire acquérir au s...  
3  Le contrat de stage doit être conclu impérativ...  
4  Le contrat de stage doit être constaté par écr...  


In [26]:
print("null : ",df.isna().sum())
df.info()

null :  Categorie    113
Tag            0
Question       0
Réponse        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Categorie  652 non-null    object
 1   Tag        765 non-null    object
 2   Question   765 non-null    object
 3   Réponse    765 non-null    object
dtypes: object(4)
memory usage: 24.0+ KB


## **2. Prétraitement des Données**

In [27]:
# Création d'une liste pour chaque question avec son tag (intention)
data = [(row['Question'], row['Tag']) for _, row in df.iterrows()]
# tags = df['Tag'].apply(str).unique()
# Récupérer les tags uniques
tags = df['Tag'].unique()
tags = df['Tag'].apply(str).unique()
print(data[:5])  # Vérifier les premières données
print(tags)  # Vérifier les tags uniques


[("Quel est l'objectif principal du contrat de stage ?", 'contrat_stage'), ("Qu'est-ce qu'un contrat de stage selon la loi ?", 'contrat_stage'), ("Quel type d'expérience le contrat de stage vise-t-il à donner au stagiaire ?", 'contrat_stage'), ('Quand doit être conclu le contrat de stage ?', 'contrat_stage'), ('Quel est le format requis pour le contrat de stage ?', 'contrat_stage')]
['contrat_stage' 'contrat_apprentissage' 'travail_temporaire_placement'
 'contrat_travail' 'contrat_essai_partielle' 'contrat_determine'
 'contrat_indetermine' 'tacheronnat' 'suspension_contrat'
 'convention_collective' 'reglement_interieur' 'duree_travail'
 'repose_hebdomadaire' 'conges' 'voyage_transport' 'salaire'
 'cautionnement' 'economat' 'securite_sante' 'syndicat_professionnel'
 'delegue_personnel' 'different_travail' 'conciliation' 'travail_femme'
 'refere' 'arbitrage' 'greve' 'inspecteur_travail' 'sanction' 'procedure'
 'contact_avocat' 'greeting' 'introduce_self' 'goodbye' 'thank_you'
 'chatbot_i

## **3. Entraînement du Modèle SpaCy**

Charger le modèle et Ajouter le Pipeline de Classification :

In [28]:
# import spacy

# spacy.cli.download("fr_core_news_md")

In [29]:
import spacy
from spacy.training import Example

# Charger un modèle vierge en français
nlp = spacy.blank("fr")

# Ajouter un pipeline de classification textuelle (intentions)
textcat = nlp.add_pipe("textcat")

# Ajouter les étiquettes (tags) pour les intentions
for tag in tags:
    textcat.add_label(tag)


In [30]:
# Préparer les données d'entraînement pour spaCy
TRAIN_DATA = []
for question, tag in data:
    # Créer les annotations pour chaque tag
    cats = {t: 1 if t == tag else 0 for t in tags}

    # Ajouter la question et ses annotations dans les données d'entraînement
    TRAIN_DATA.append((question, {"cats": cats}))

# Afficher quelques exemples des données préparées
print(TRAIN_DATA[:3])


[("Quel est l'objectif principal du contrat de stage ?", {'cats': {'contrat_stage': 1, 'contrat_apprentissage': 0, 'travail_temporaire_placement': 0, 'contrat_travail': 0, 'contrat_essai_partielle': 0, 'contrat_determine': 0, 'contrat_indetermine': 0, 'tacheronnat': 0, 'suspension_contrat': 0, 'convention_collective': 0, 'reglement_interieur': 0, 'duree_travail': 0, 'repose_hebdomadaire': 0, 'conges': 0, 'voyage_transport': 0, 'salaire': 0, 'cautionnement': 0, 'economat': 0, 'securite_sante': 0, 'syndicat_professionnel': 0, 'delegue_personnel': 0, 'different_travail': 0, 'conciliation': 0, 'travail_femme': 0, 'refere': 0, 'arbitrage': 0, 'greve': 0, 'inspecteur_travail': 0, 'sanction': 0, 'procedure': 0, 'contact_avocat': 0, 'greeting': 0, 'introduce_self': 0, 'goodbye': 0, 'thank_you': 0, 'chatbot_info': 0, 'droit_foncier': 0, 'droit_civil': 0}}), ("Qu'est-ce qu'un contrat de stage selon la loi ?", {'cats': {'contrat_stage': 1, 'contrat_apprentissage': 0, 'travail_temporaire_placement

Entraîner le Modèle avec les Données :

In [31]:
# # Entraînement du modèle avec spaCy
# Initialisation de l'optimiseur pour le modèle
optimizer = nlp.begin_training()

# Nombre d'itérations (epochs) pour l'entraînement
n_iter = 10

# Boucle sur chaque itération d'entraînement
for itn in range(n_iter):
    # Dictionnaire pour stocker les pertes (losses) pendant cette itération
    losses = {}

    # Boucle sur les données d'entraînement
    for text, annotations in TRAIN_DATA:
        # Vérification si 'text' est une chaîne de caractères et non 'nan'
        if isinstance(text, str) and text != 'nan':
            # Création d'un document spaCy à partir du texte
            doc = nlp.make_doc(text)
            # Création d'un exemple à partir du document et des annotations
            example = Example.from_dict(doc, annotations)
            # Mise à jour du modèle avec l'exemple, en appliquant un taux de dropout de 0.2
            nlp.update([example], drop=0.2, losses=losses)

    # Affichage des pertes pour l'itération actuelle
    print(f"Iteration {itn + 1} - Losses: {losses}")


Iteration 1 - Losses: {'textcat': 16.644941830257608}
Iteration 2 - Losses: {'textcat': 15.128939696136344}
Iteration 3 - Losses: {'textcat': 14.445558245138496}
Iteration 4 - Losses: {'textcat': 13.458035957314637}
Iteration 5 - Losses: {'textcat': 11.142921068919627}
Iteration 6 - Losses: {'textcat': 9.001319967436693}
Iteration 7 - Losses: {'textcat': 7.318186637885226}
Iteration 8 - Losses: {'textcat': 5.887898654656514}
Iteration 9 - Losses: {'textcat': 5.57188089901807}
Iteration 10 - Losses: {'textcat': 4.417012272207533}


In [32]:
# Sauvegarder le modèle entraîné
nlp.to_disk("/content/modele_veenge_maan_chatbot_juridique")

# Télécharger le modèle sur ton local depuis Colab
from google.colab import files
import shutil

shutil.make_archive("modele_veenge_maan_chatbot_juridique", 'zip', "/content/modele_veenge_maan_chatbot_juridique")
files.download("modele_veenge_maan_chatbot_juridique.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **4. Évaluation du Modèle**

In [33]:
test_df = pd.read_csv("/content/tester_modele_spacy.csv")
test_df = test_df[["Tag", "Question", "Réponse"]]
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tag       782 non-null    object
 1   Question  782 non-null    object
 2   Réponse   782 non-null    object
dtypes: object(3)
memory usage: 18.5+ KB


In [34]:

from sklearn.metrics import classification_report

# Fonction pour évaluer le modèle SpaCy
def evaluate_model(nlp, valid_data):
    predictions = []
    true_labels = []

    for text, annotations in valid_data:
        # Prédire le tag de la question
        doc = nlp(text)
        predicted_label = max(doc.cats, key=doc.cats.get)

        # Extraire le tag réel depuis les annotations
        true_label = max(annotations["cats"], key=annotations["cats"].get)

        predictions.append(predicted_label)
        true_labels.append(true_label)

    # Afficher le rapport de classification
    print(classification_report(true_labels, predictions))

# Exécuter la fonction d'évaluation l'ensemble de validation
valid_data = TRAIN_DATA[:]

# Exécuter la fonction d'évaluation sur l'ensemble de test
evaluate_model(nlp, valid_data)


                              precision    recall  f1-score   support

                   arbitrage       0.80      1.00      0.89         8
               cautionnement       1.00      0.87      0.93        15
                chatbot_info       0.93      1.00      0.96        13
                conciliation       1.00      0.93      0.97        15
                      conges       0.85      0.65      0.73        17
              contact_avocat       1.00      0.95      0.97        40
       contrat_apprentissage       0.89      0.89      0.89         9
           contrat_determine       1.00      0.67      0.80        12
     contrat_essai_partielle       1.00      1.00      1.00         7
         contrat_indetermine       1.00      0.34      0.51        32
               contrat_stage       1.00      1.00      1.00         8
             contrat_travail       1.00      0.40      0.57        15
       convention_collective       1.00      0.87      0.93        76
           delegue_

In [35]:
#  un ensemble de test
test_df = pd.read_csv("/content/tester_modele_spacy.csv")
test_df = test_df[["Tag", "Question", "Réponse"]]
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tag       782 non-null    object
 1   Question  782 non-null    object
 2   Réponse   782 non-null    object
dtypes: object(3)
memory usage: 18.5+ KB


In [36]:
#  un ensemble de test

# Préparer les données de test
y_true = []
y_pred = []

for _, row in test_df.iterrows():
    text = row['Question']
    true_tag = row['Tag']
    y_true.append(true_tag)

    # Prédiction du modèle
    doc = nlp(text)
    predicted_tag = doc.cats
    # Choisir le tag avec la probabilité maximale
    predicted_tag = max(predicted_tag, key=predicted_tag.get)
    y_pred.append(predicted_tag)

# Calcul des métriques
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
unique_labels = set(y_true)
predicted_labels = set(y_pred)

# Étiquettes qui n'ont pas été prédites
missing_labels = unique_labels - predicted_labels
print(f"Étiquettes non prédites : {missing_labels}")

print(f"Précision: {precision:.2f}")
print(f"Rappel: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Exactitude: {accuracy:.2f}")


Étiquettes non prédites : set()
Précision: 0.90
Rappel: 0.85
F1-score: 0.84
Exactitude: 0.85


## **5. Sauvegarde et Chargement du Modèle**

In [24]:
# Sauvegarder le modèle entraîné
nlp.to_disk("/content/modele_veenge_maan_chatbot_juridique")

# Télécharger le modèle sur ton local depuis Colab
from google.colab import files
import shutil

shutil.make_archive("modele_veenge_maan_chatbot_juridique", 'zip', "/content/modele_veenge_maan_chatbot_juridique")
files.download("modele_veenge_maan_chatbot_juridique.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>